In [1]:
from sqlalchemy import create_engine, ForeignKey, MetaData
from sqlalchemy import Table, Column, Date, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import os
import pandas as pd

In [2]:
database_path = os.path.join('Retail_Industry_Health_DB.sqlite')
loadpath = os.path.join('.')
naics_load = os.path.join(loadpath,'naics.csv')


In [3]:
# Establishe a link to the sqlite database (creating it if necessary)
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()
Base = declarative_base()

# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [4]:

conn.execute('DROP TABLE IF EXISTS NAICS_Info;')
conn.execute('CREATE TABLE NAICS_Info (NAICS_Code VARCHAR(15) NOT NULL,Industry_Title VARCHAR(150) NOT NULL, Num_Businesses BIGINT  NOT NULL);')

conn.execute('DROP TABLE IF EXISTS Employement;')
conn.execute('CREATE TABLE Employement (NAICS_Code VARCHAR(15) NOT NULL,Year INT,Month VARCHAR(3),Num_Employees FLOAT);')

conn.execute('DROP TABLE IF EXISTS Sales;')
conn.execute('CREATE TABLE Sales (NAICS_Code VARCHAR(15) NOT NULL,Year INT,Month VARCHAR(3),Total_Sales FLOAT);')

session.commit()

In [5]:
naics = pd.read_csv('naics.csv')
naics.to_sql('NAICS_Info', con=engine, if_exists='append', index = False, index_label=None)
session.commit()
Base.metadata.create_all(conn)

In [6]:
test = pd.read_sql("SELECT * FROM NAICS_Info", engine)
test_df = pd.DataFrame(test)
test_df.head()

,NAICS_Code,Industry_Title,Num_Businesses
0,11,"Agriculture, Forestry, Fishing and Hunting",350038
1,1111,Oilseed and Grain Farming,55359
2,111110,Soybean Farming,4590
3,111120,Oilseed (except Soybean) Farming,154
4,111130,Dry Pea and Bean Farming,3430


In [7]:
emp = pd.read_csv('final_emp_data.csv')
emp.to_sql('Employement', con=engine, if_exists='append', index = False, index_label=None)
session.commit()
Base.metadata.create_all(conn)

In [8]:
test2 = pd.read_sql("SELECT * FROM Employement", engine)
test2_df = pd.DataFrame(test2)
test2_df.head()

,NAICS_Code,Year,Month,Num_Employees
0,441,2009,Jan,1368.9
1,441,2010,Jan,1301.6
2,441,2011,Jan,1344.5
3,441,2012,Jan,1384.4
4,441,2013,Jan,1428.5


In [9]:
sales_data = pd.read_csv('sales_data.csv')
sales_data.to_sql('Sales', con=engine, if_exists='append', index = False, index_label=None)
session.commit()
Base.metadata.create_all(conn)

In [10]:
test3 = pd.read_sql("SELECT * FROM Sales", engine)
test3_df = pd.DataFrame(test3)
test3_df.head()

,NAICS_Code,Year,Month,Total_Sales
0,441,2018,Jan,87735.0
1,"4411,4412",2018,Jan,80723.0
2,4411,2018,Jan,76671.0
3,44111,2018,Jan,67953.0
4,44112,2018,Jan,8718.0


In [11]:
session.commit()

In [12]:
session.close()

In [13]:
conn.close()